In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append("../")
import biobss

In [2]:
import neurokit2 as nk
sample_data = pd.read_pickle("..\sample_data\\EDA_Chest.pkl")
sample_data = sample_data.flatten()

In [3]:
biobss.pipelinev2

<module 'biobss.pipelinev2' from 'c:\\Users\\Cagatay\\Desktop\\Biobss\\biobss_pipeline_refactor\\biobss\\examples\\..\\biobss\\pipelinev2\\__init__.py'>

In [4]:
from biobss import pipelinev2 as pipeline

In [5]:
decompose = pipeline.Bio_Process(
    process_method=biobss.edatools.eda_decompose, modality="EDA", sigtype="EDA", method="highpass")

filtering=pipeline.Bio_Process(
    process_method=biobss.signaltools.filter_signal, modality="EDA", sigtype="EDA",N=2, filter_type="highpass",f1=0.5,fs=700)

normalize = pipeline.Bio_Process(
    process_method=biobss.signaltools.normalize_signal, modality="EDA", sigtype="EDA")
resample = pipeline.Bio_Process(
    process_method=biobss.signaltools.resample_signal_object, modality="EDA", sigtype="EDA", target_sample_rate=350)
signal_features = pipeline.Feature(name="signal_features", function=biobss.edatools.signal_features.get_signal_features, parameters={
                                           "modality": "EDA", "sigtype": "EDA"}, input_signals={'EDA_Raw':'EDA_Raw', 'EDA_Tonic':'EDA_Tonic', 'EDA_Phasic':'EDA_Phasic'})
stat_features = pipeline.Feature(name="stat_features", function=biobss.common.stat_features.get_stat_features, parameters={
                                         "modality": "EDA", "sigtype": "EDA"}, input_signals={'EDA_Raw':'EDA_Raw', 'EDA_Tonic':'EDA_Tonic', 'EDA_Phasic':'EDA_Phasic'})
corr_features = pipeline.Feature(name="corr_features", function=biobss.common.correlation_features, parameters={
                                        "modality": "EDA", "sigtype": "EDA","signal_names":['EDA_Raw','EDA_Tonic','EDA_Phasic']}, input_signals={'EDA':['EDA_Raw','EDA_Tonic','EDA_Phasic']})

In [6]:
pipe = pipeline.Bio_Pipeline(windowed_process=True,window_size=60, step_size=30, features_list=[])


In [7]:
pipe.preprocess_queue.add_process(normalize)
pipe.process_queue.add_process(resample)
pipe.process_queue.add_process(decompose)
pipe.add_feature_step(signal_features)
pipe.add_feature_step(stat_features)
pipe.add_feature_step(corr_features)


In [8]:
pipe.set_input(sample_data,sampling_rate=256)

ValueError: If signal is not a Bio_Data object, sampling_rate must be specified